# Demonstration Notebook
## Improvise melody based on learned musical style

By Ben Walsh \
For Liloquy

&copy; 2021 Ben Walsh <ben@liloquy.io>

## Contents

1. [Import Libraries](#lib_import)
1. [Define Model](#model_define)
1. [Test Model](#model_test)
1. [Generate Melody](#gen_melody)
1. [Play Melody](#play_melody)

TO DO
- Playback generated melody - expand single note to full melody. See liloquy-git melody.py
- Move normalize_distro to util subfolder and import

## <a id = "lib_import"></a>1. Import Libraries

In [1]:
import numpy as np
import sys
from pomegranate import State, HiddenMarkovModel, DiscreteDistribution

# Custom libraries

# Add custom modules to path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# from util import normalize_distro
from util.music_util import gen_melody_from_distro, Note

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


## <a id = "model_define"></a>2. Define Model

### Define (placeholder) initial note distributions for two genres

In [2]:
note_distro = {}
note_distro['major'] = {
    'C4': 0.3,
    'D4': 0.1,
    'E4': 0.2,
    'F4': 0.05,
    'G4': 0.2,
    'A4': 0.1,
    'B4': 0.05
}

note_distro['minor'] = {
    'C4': 0.2,
    'D4': 0.1,
    'E4': 0.2,
    'F4': 0.05,
    'G4': 0.1,
    'A4': 0.3,
    'B4': 0.05
}

In [3]:
note_distro

{'major': {'C4': 0.3,
  'D4': 0.1,
  'E4': 0.2,
  'F4': 0.05,
  'G4': 0.2,
  'A4': 0.1,
  'B4': 0.05},
 'minor': {'C4': 0.2,
  'D4': 0.1,
  'E4': 0.2,
  'F4': 0.05,
  'G4': 0.1,
  'A4': 0.3,
  'B4': 0.05}}

In [4]:
genre_distro_start = {}
genre_distro_start = {
    'major': 0.5,
    'minor': 0.5
}

genre_distro_end = {}
genre_distro_end = {
    'major': 0.5,
    'minor': 0.5
}

genre_distro_trans = {
    ('major', 'major'): 0.9,
    ('major', 'minor'): 0.1,
    ('minor', 'major'): 0.2,
    ('minor', 'minor'): 0.8,
}

### Define Hidden Markov Model using Pomegranate

In [5]:
# Initialize model
hmm_model = HiddenMarkovModel(name="hmm-note-transitions")

# Initialize the states in the HMM
states = {}

# Add initial distributions
for genre in note_distro.keys():
    
    # Ensure normalization
    # note_distro[genre] = normalize_distro(note_distro_genre)
    sum_values = sum(note_distro[genre].values())
    for k, v in note_distro[genre].items():
        note_distro[genre].update({k:v/sum_values})
    
    # Define HMM state for genre
    states[genre] = State(DiscreteDistribution(note_distro[genre]), name=genre)
    
    # Add state to hmm_model
    hmm_model.add_states(states[genre])

# Add starting genre distributions
for genre in note_distro.keys():
    hmm_model.add_transition(hmm_model.start, states[genre], genre_distro_start[genre])

# Add ending note distributions
for genre in note_distro.keys():
    hmm_model.add_transition(states[genre], hmm_model.end, genre_distro_end[genre])

for (genre1, genre2) in genre_distro_trans.keys():
    hmm_model.add_transition(states[genre1], states[genre2], genre_distro_trans[(genre1, genre2)])

hmm_model.bake()

## <a id = "model_test"></a>3. Test Model 

### Predict genre on input melodies

In [6]:
major_seq = ('C4', 'E4', 'G4', 'C4', 'E4', 'G4')
[hmm_model.states[genre].name for genre in hmm_model.predict(major_seq)]

['major', 'major', 'major', 'major', 'major', 'major']

In [7]:
minor_seq = ('A4', 'C4', 'E4', 'A4', 'C4', 'E4', 'A4')
[hmm_model.states[genre].name for genre in hmm_model.predict(minor_seq)]

['minor', 'minor', 'minor', 'minor', 'minor', 'minor', 'minor']

## <a id = "gen_melody"></a>4. Generate Melody

In [8]:
genre_to_test = 'major'
test_melody_length = 8

In [9]:
generated_melody = gen_melody_from_distro(note_distro[genre_to_test], melody_len=test_melody_length)
print(generated_melody)

['E4', 'D4', 'E4', 'G4', 'C4', 'G4', 'D4', 'C4']


In [10]:
# Apply hmm_model to estimate the genre
[hmm_model.states[genre].name for genre in hmm_model.predict(generated_melody)]

['major', 'major', 'major', 'major', 'major', 'major', 'major', 'major']

## <a id = "play_melody"></a>5. Playback Melody

In [12]:
note_to_play = Note(note=generated_melody[0])
note_to_play.sound.play(1)